In [ ]:
import pandas as pd
import os
import numpy as np
from nilearn.maskers import NiftiSpheresMasker
from nilearn.maskers import NiftiMasker
from nilearn import image

In [ ]:
# Import language network mask

neurosynth_file = '{}masks/language_network_MNIasym_symetric.nii.gz'.format(project_path)
nimg = image.load_img(neurosynth_file)
mask_data = nimg.get_fdata()

In [ ]:
# Define maskers

coords = [(44, 28, 5)]

seed_masker = NiftiSpheresMasker(
    coords,
    radius=10,
    detrend=True,
    standardize=True,
    t_r=2,
    memory="nilearn_cache",
    memory_level=1,
    verbose=0,
)

brain_masker = NiftiMasker(
    smoothing_fwhm=6,
    detrend=True,
    standardize=True,
    t_r=2,
    memory="nilearn_cache",
    memory_level=1,
    verbose=0,
)

In [ ]:
# Internal dataset processing
# Load dataset

project_path = '/Users/labneuro2/Documents/lab/RS_LI/'
df = pd.read_excel('{}internal_LI_indexes.xlsx'.format(project_path))
subs = sorted(df['Subject'].unique())

#Seed to voxel correlation maps:

internal_seed_corr_dir = '{}internal_seed_corr/'.format(project_path)
if os.path.isdir(internal_seed_corr_dir)==False:
      os.mkdir(internal_seed_corr_dir)
        
internal_input_dir = '{}internal_input_features/'.format(project_path)
if os.path.isdir(internal_input_dir)==False:
      os.mkdir(internal_input_dir)

        
func_path = '{}internal_halfpipe/'.format(project_path)
for sub in subs:
    for ses in ['ses-1','ses-2']: 
        func_filename = '{}{}/{}/func/{}_{}_task-rest_setting-preproc_bold.nii.gz'.format(func_path,sub,ses,sub,ses)
        
        # Compute correlations
        
        seed_time_series = seed_masker.fit_transform(func_filename)
        brain_time_series = brain_masker.fit_transform(func_filename)
        seed_to_voxel_correlations = (np.dot(brain_time_series.T, seed_time_series) / seed_time_series.shape[0])
        seed_to_voxel_correlations_fisher_z = np.arctanh(seed_to_voxel_correlations)
        seed_to_voxel_corr_img = brain_masker.inverse_transform(seed_to_voxel_correlations_fisher_z.T)
        
         # Save correlation image

        img_path = '{}{}_{}_seed_to_voxel_corr.nii.gz'.format(internal_seed_corr_dir,sub,ses)
        seed_to_voxel_corr_img.to_filename(img_path)
        
        # Prepare 2D images
        
        new_data = np.zeros((3,97,115))
        img = image.load_img(img_path)
        corr_data = img.get_fdata()
        corr_data[mask_data == 0] = 0
        data = np.mean(corr_data[:,:,30:50], axis=2)
        for k in range(3):
            new_data[k,:,:]  = data[:,:,0]
        np.save('{}{}_{}.npy'.format(internal_input_dir,sub,ses), new_data)  

In [ ]:
# External dataset processing
# Load dataset

df = pd.read_excel('{}external_LI_indexes.xlsx'.format(project_path))
subs = sorted(df['Subject'].unique())

#Seed to voxel correlation maps:

external_seed_corr_dir = '{}external_seed_corr/'.format(project_path)
if os.path.isdir(external_seed_corr_dir)==False:
      os.mkdir(external_seed_corr_dir)
        
external_input_dir = '{}external_input_features/'.format(project_path)
if os.path.isdir(external_input_dir)==False:
      os.mkdir(external_input_dir)

        
func_path = '{}external_halfpipe/'.format(project_path)
for sub in subs:
    func_filename = '{}sub-{}/func/sub-{}_task-rest_setting-preproc_bold.nii.gz'.format(func_path,sub,sub)
    
    # Compute correlations
        
    seed_time_series = seed_masker.fit_transform(func_filename)
    brain_time_series = brain_masker.fit_transform(func_filename)
    seed_to_voxel_correlations = (np.dot(brain_time_series.T, seed_time_series) / seed_time_series.shape[0])
    seed_to_voxel_correlations_fisher_z = np.arctanh(seed_to_voxel_correlations)
    seed_to_voxel_corr_img = brain_masker.inverse_transform(seed_to_voxel_correlations_fisher_z.T)
    
     # Save correlation image
        
    img_path = '{}{}_seed_to_voxel_corr.nii.gz'.format(external_seed_corr_dir,sub)
    seed_to_voxel_corr_img.to_filename(img_path)
    
     # Prepare 2D images
        
    new_data = np.zeros((3,97,115))
    img = image.load_img(img_path)
    corr_data = img.get_fdata()
    corr_data[mask_data == 0] = 0
    data = np.mean(corr_data[:,:,30:50], axis=2)
    for k in range(3):
        new_data[k,:,:]  = data[:,:,0]
    np.save('{}{}.npy'.format(external_input_dir,sub), new_data) 